In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/matching/1.xlsx
/kaggle/input/matching/2.xlsx


In [2]:
from sklearn import feature_extraction, metrics
import io
from tqdm import tqdm

In [3]:
class StringMatcher():
    
    def __init__(self, dtf_left, dtf_right):
        self.dtf_left = dtf_left
        self.dtf_right = dtf_right
    
    
    @staticmethod
    def utils_string_matching(a, lst_b, threshold=None, top=None):
        ## vectorizer ("my house" --> ["my", "hi", "house", "sky"] --> [1, 0, 1, 0])
        vectorizer = feature_extraction.text.CountVectorizer()
        X = vectorizer.fit_transform([a]+lst_b).toarray()

        ## cosine similarity (scores a vs lst_b)
        lst_vectors = [vec for vec in X]
        cosine_sim = metrics.pairwise.cosine_similarity(lst_vectors)
        scores = cosine_sim[0][1:]

        ## match
        match_scores = scores if threshold is None else scores[scores >= threshold]
        match_idxs = range(len(match_scores)) if threshold is None else [i for i in np.where(scores >= threshold)[0]] 
        match_strings = [lst_b[i] for i in match_idxs]

        ## dtf
        dtf_match = pd.DataFrame(match_scores, columns=[a], index=match_strings)
        dtf_match = dtf_match[~dtf_match.index.duplicated(keep='first')].sort_values(a, ascending=False).head(top)
        return dtf_match
    
    
    def vlookup(self, threshold=0.7, top=1):
        ## process data
        lst_left = list(set( self.dtf_left.iloc[:,0].tolist() ))
        lst_right = list(set( self.dtf_right.iloc[:,0].tolist() ))
        
        ## match strings
        dtf_matches = pd.DataFrame(columns=['string','match','similarity'])
        for string in tqdm(lst_left):
            dtf_match = self.utils_string_matching(string, lst_right, threshold, top)
            dtf_match = dtf_match.reset_index().rename(columns={'index':'match', string:'similarity'})
            dtf_match["string"] = string
            dtf_matches = dtf_matches.append(dtf_match, ignore_index=True, sort=False)
        return dtf_matches[['string','match','similarity']]
    
    
    @staticmethod
    def write_excel(dtf):
        bytes_file = io.BytesIO()
        excel_writer = pd.ExcelWriter(bytes_file)
        dtf.to_excel(excel_writer, sheet_name='Sheet1', na_rep='', index=False)
        excel_writer.save()
        bytes_file.seek(0)
        return bytes_file

In [4]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 4.0 MB/s eta 0:00:00


In [5]:
dtf_left = pd.read_excel("../input/matching/1.xlsx")
dtf_right = pd.read_excel("../input/matching/2.xlsx")

In [6]:
match_ = StringMatcher(dtf_left, dtf_right)

In [7]:
match_2 = match_.vlookup(threshold = 0.5, top = 3)

100%|██████████| 429/429 [02:02<00:00,  3.51it/s]


In [8]:
match_2.head()

,string,match,similarity
0,ASAD AYAZ SHAIKH,~Asad Shaikh,0.816497
1,OKSANA FJODOROVA,OKSANA FJODOROVA,1.000000
2,Piquois Gisèle,Piquois Gisèle,1.000000
3,YLKA FIGUEREDO DE OLVEIRA,Ylka Figueredo,0.707107
4,Lina Jalouk,Lina Jalouk,1.000000


In [9]:
match_2.to_excel("matchPoint5.xlsx")

In [10]:
match_2.shape

(392, 3)